# Tutorial 1: Matching

## 0. Import libraries 

In [1]:
import pandas as pd
import numpy as np
from hypex import Matcher
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
from hypex.utils.tutorial_data_creation import create_test_data

In [3]:
df = create_test_data(rs=42, na_step=45, nan_cols=['age', 'gender'])
df

,user_id,signup_month,treat,pre_spends,post_spends,age,gender,industry
0,0,0,0,504.5,422.777778,NaN,F,Logistics
1,2,0,0,485.0,434.000000,34.0,NaN,E-commerce
2,4,0,0,488.5,420.111111,33.0,F,E-commerce
3,5,0,0,475.0,408.333333,52.0,M,E-commerce
4,6,0,0,494.0,423.666667,33.0,M,Logistics
...,...,...,...,...,...,...,...,...
5468,9992,0,0,456.5,404.888889,34.0,M,E-commerce
5469,9993,0,0,503.0,431.888889,64.0,F,Logistics
5470,9994,0,0,480.0,411.444444,67.0,F,E-commerce
5471,9996,0,0,453.0,406.888889,27.0,M,Logistics


In [4]:
df.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'gender', 'industry'],
      dtype='object')

In [5]:
df['treat'].value_counts()

0    5002
1     471
Name: treat, dtype: int64

In [6]:
df['gender'].isna().sum()

122

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [7]:
info_col = ['user_id']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [8]:
# Standard model with base parameters
model = Matcher(input_data=df, outcome=outcome, treatment=treatment, info_col=info_col)

[23.01.2024 09:16:29 | hypex | INFO]: Number of NaN values filled with zeros: 244


In [9]:
selected_features = model.feature_select()
selected_features

,rank
signup_month,1
pre_spends,2
age,3
gender_F,4
gender_M,5
industry_Logistics,6


In [10]:
chosen_features = selected_features[:4].index
chosen_features

Index(['signup_month', 'pre_spends', 'age', 'gender_F'], dtype='object')

In [11]:
results, quality_results, df_matched = model.estimate(features=chosen_features)

  0%|          | 0/5473 [00:00<?, ?it/s]

In [12]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,-322.294666,0.530585,0.0,-323.334613,-321.254719,post_spends
ATC,-362.128225,0.540184,0.0,-363.186985,-361.069465,post_spends
ATT,100.736041,0.641382,0.0,99.478932,101.993150,post_spends


In [13]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [14]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000e+00,0.129535
pre_spends,1.000000e+00,0.760687
signup_month,2.070021e-282,0.000000


In [15]:
df_matched

,index,signup_month,pre_spends,age,gender_F,gender_M,industry_Logistics,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_Logistics_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,32,3,481.5,63.0,0,1,0,0.0,481.500000,63.000000,0.0,1.0,0.500000,"[5880, 5938]",511.333333,414.000000,97.333333,1,0
1,38,3,493.0,20.0,0,1,0,0.0,492.500000,20.000000,0.0,1.0,0.000000,[323],519.666667,402.111111,117.558440,1,0
2,92,3,511.0,40.0,1,0,1,0.0,511.000000,39.000000,1.0,0.0,0.000000,[9665],535.777778,426.666667,109.060675,1,0
3,117,3,478.0,56.0,1,0,1,0.0,478.000000,56.000000,1.0,0.0,0.000000,[4243],529.000000,415.000000,114.000000,1,0
4,119,3,470.0,57.0,1,0,1,0.0,470.666667,57.333333,1.0,0.0,0.333333,"[280, 2931, 5456]",523.444444,413.111111,110.346300,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997,9992,0,456.5,34.0,0,1,0,3.0,463.000000,31.000000,0.0,1.0,0.000000,[9179],404.888889,515.555556,-351.957566,0,1
4998,9993,0,503.0,64.0,1,0,1,3.0,500.500000,63.000000,1.0,0.0,1.000000,[5564],431.888889,521.111111,-373.248147,0,1
4999,9994,0,480.0,67.0,1,0,0,3.0,483.000000,64.000000,1.0,0.0,0.000000,[9225],411.444444,514.222222,-359.795854,0,1
5000,9996,0,453.0,27.0,0,1,1,3.0,456.000000,23.000000,0.0,1.0,1.000000,[6115],406.888889,518.000000,-351.474394,0,1


In [16]:
df_matched[df_matched['industry_Logistics'] != df_matched['industry_Logistics_matched']]

,index,signup_month,pre_spends,age,gender_F,gender_M,industry_Logistics,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_Logistics_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,32,3,481.5,63.0,0,1,0,0.0,481.500000,63.000000,0.0,1.0,0.500000,"[5880, 5938]",511.333333,414.000000,97.333333,1,0
2,92,3,511.0,40.0,1,0,1,0.0,511.000000,39.000000,1.0,0.0,0.000000,[9665],535.777778,426.666667,109.060675,1,0
3,117,3,478.0,56.0,1,0,1,0.0,478.000000,56.000000,1.0,0.0,0.000000,[4243],529.000000,415.000000,114.000000,1,0
4,119,3,470.0,57.0,1,0,1,0.0,470.666667,57.333333,1.0,0.0,0.333333,"[280, 2931, 5456]",523.444444,413.111111,110.346300,1,0
6,143,3,510.0,66.0,1,0,0,0.0,509.500000,66.000000,1.0,0.0,1.000000,[7607],517.000000,437.888889,79.113995,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,9975,0,489.5,21.0,1,0,1,3.0,489.500000,20.000000,1.0,0.0,0.000000,[7305],418.111111,519.222222,-361.395402,0,1
4993,9980,0,467.5,53.0,0,1,0,3.0,468.500000,50.000000,0.0,1.0,1.000000,[1711],415.555556,526.666667,-351.433606,0,1
4994,9981,0,458.0,23.0,0,1,0,3.0,456.000000,23.000000,0.0,1.0,1.000000,[6115],422.666667,518.000000,-367.132391,0,1
4996,9986,0,486.5,19.0,0,1,1,3.0,485.500000,22.000000,0.0,1.0,0.000000,[4334],418.111111,527.000000,-353.555673,0,1


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [17]:
group_col = "industry"

In [18]:
model = Matcher(input_data=df, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

[23.01.2024 09:16:32 | hypex | INFO]: Number of NaN values filled with zeros: 244


In [19]:
selected_features = model.feature_select()
selected_features

,rank
signup_month,1
pre_spends,2
age,3
gender_F,4
gender_M,5


In [20]:
chosen_features = selected_features[:4].index
chosen_features

Index(['signup_month', 'pre_spends', 'age', 'gender_F'], dtype='object')

In [21]:
results, quality_results, df_matched = model.estimate(features=chosen_features)

  0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,-316.075532,0.531084,0.0,-317.116456,-315.034609,post_spends
ATC,-355.282996,0.541203,0.0,-356.343754,-354.222239,post_spends
ATT,100.306066,0.642753,0.0,99.046271,101.565862,post_spends


In [23]:
df_matched[df_matched['industry'] != df_matched['industry_matched']]

,index,signup_month,pre_spends,age,gender_F,gender_M,industry,signup_month_matched,pre_spends_matched,age_matched,gender_F_matched,gender_M_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched


## 3. Results  
### 3.1 ATE, ATT, ATC

In [24]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,-316.075532,0.531084,0.0,-317.116456,-315.034609,post_spends
ATC,-355.282996,0.541203,0.0,-356.343754,-354.222239,post_spends
ATT,100.306066,0.642753,0.0,99.046271,101.565862,post_spends


### 3.2 SMD, PSI, KS-test, repeats

In [25]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [26]:
quality_results['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,gender_F_treated,0.00,OK,gender_F_untreated,0.00,OK
2,industry_treated,0.00,OK,industry_untreated,0.00,OK
3,pre_spends_treated,0.01,OK,pre_spends_untreated,0.01,OK
4,signup_month_treated,18.42,NOK,signup_month_untreated,0.00,OK


In [27]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000e+00,0.087975
pre_spends,1.000000e+00,0.170224
signup_month,2.070021e-282,0.000000


In [28]:
quality_results['repeats']

{'match_control_to_treat': 0.95, 'match_treat_to_control': 0.09}

### 3.3 Validation
Validates estimated effect:
1. by replacing real treatment (`random_treatment`) with random placebo treatment.
 Estimated effect must be droped to zero;
2. by adding random feature (`random_feature`). Estimated effect shouldn't change
significantly, p-val < 0.05;
3. estimates effect on subset of data (`subset_refuter`) (default fraction is 0.8). Estimated effect
shouldn't change significantly, p-val < 0.05.

In [29]:
model.validate_result(refuter="random_treatment", effect_type="att", n_sim=10)

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [-0.07194058875830694, 0.0]}

## 4. Save model

In [30]:
model.save("test_model.pickle")

In [31]:
model2 = Matcher.load("test_model.pickle")

In [32]:
model2.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,-316.075532,0.531084,0.0,-317.116456,-315.034609,post_spends
ATC,-355.282996,0.541203,0.0,-356.343754,-354.222239,post_spends
ATT,100.306066,0.642753,0.0,99.046271,101.565862,post_spends


In [33]:
model.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,-316.075532,0.531084,0.0,-317.116456,-315.034609,post_spends
ATC,-355.282996,0.541203,0.0,-356.343754,-354.222239,post_spends
ATT,100.306066,0.642753,0.0,99.046271,101.565862,post_spends
